In [ ]:
# !pip install --upgrade --quiet google-cloud-aiplatform

In [1]:
import vertexai
from IPython.display import display, Markdown
from vertexai.generative_models import (
    FunctionDeclaration,
    GenerativeModel,
    GenerationConfig,
    Part,
    Tool,
    GenerationResponse
)

from typing import Any, Dict, List
from google.cloud import bigquery

PROJECT_ID = "no-latency-labs"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)

client = bigquery.Client()

query_product_info = FunctionDeclaration(
    name="query_product_info",
    description="Fetch the product info from a bigquery table",
    parameters={
        "type": "object",
        "properties": {
            "product_name": {
                "type": "string",
                "description": "Name of the product for which the details are to be fetched",
            }
        },
    },
)

get_weekly_product_orders = FunctionDeclaration(
    name="get_weekly_product_orders",
    description="Fetch the total orders for the last 1 week for any given product",
    parameters={
        "type": "object",
        "properties": {
           "product_name": {
                "type": "string",
                "description": "Name of the product for which the details are to be fetched",
            }
        },
    },
)

fetch_info_tool = Tool(
    function_declarations=[query_product_info,get_weekly_product_orders],
)

gemini_model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[fetch_info_tool],
)

In [2]:
def query_product_info(product_name):

    query = f"""
    SELECT distinct name, brand, ROUND(retail_price, 1) AS price, department
    FROM `bigquery-public-data.thelook_ecommerce.products`
    WHERE LOWER(name) LIKE @product_name
    """

    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("product_name", "STRING", f"%{product_name.lower()}%")
        ]
    )

    # Run the query
    query_job = client.query(query, job_config=job_config)

    # Wait for the query to finish
    results = query_job.result()

    # Process the results
    response = []
    for row in results:
        response.append({
            "name": row.name,
            "brand": row.brand,
            "price": row.price,
            "department": row.department
        })

    return response

def get_weekly_product_orders(product_name):

    product_name = product_name.lower().strip()
    query = f"""
        select a.name as product_name,b.total_orders from 
        (SELECT id,name,brand,round(retail_price,1) as price,department FROM `bigquery-public-data.thelook_ecommerce.products` 
        where lower(name) like '%{product_name}%') a
        join 
        (
          SELECT product_id,sum(distinct order_id) as total_orders FROM `bigquery-public-data.thelook_ecommerce.order_items` 
        where date(created_at)>=current_date-7 group by 1
        ) b
        on a.id = b.product_id
    """
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("product_name", "STRING", f"%{product_name.lower()}%")
        ]
    )

    # Run the query
    query_job = client.query(query, job_config=job_config)

    # Wait for the query to finish
    results = query_job.result()
    
    results = list(query_job.result()) 
    
    if results is None or not results:
        response = [{'product_name':product_name, 'total_orders': 0}]
    else:
        response = []
        for row in results:
            response.append({
                "product_name": row.product_name,
                "total_orders": row.total_orders
            })

    return response

In [45]:
chat = gemini_model.start_chat()

prompt = """Whats the cost of Low Profile Dyed Cotton Twill Cap - Navy W39S55D 
        and how many orders were place for it in the past week?
        """

# prompt = """How many orders were placed for Enzyme Regular Solid Army Caps-Black W35S45D in the past week?"""

prompt += """
    Give a concise, high-level summary. 
    Only use information that you learn from BigQuery depending on the question and do not make up any information.
    """

response = chat.send_message(prompt)

In [47]:
# response

In [48]:
function_name = response.candidates[0].content.parts[0].function_call.name

for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    arg_name = key
    arg_value = value.lower().strip()

final_response = ''
if function_name == 'query_product_info':
    final_response = query_product_info(arg_value)

if function_name == 'get_weekly_product_orders':
    final_response = get_weekly_product_orders(arg_value)

In [49]:
final_response

[{'name': 'Low Profile Dyed Cotton Twill Cap - Navy W39S55D',
  'brand': 'MG',
  'price': 6.3,
  'department': 'Women'}]

In [19]:
response = chat.send_message(
    Part.from_function_response(
        name=function_name,
        response={
            "content": final_response,
        },
    ),
)
response.text

'There were no orders for Enzyme Regular Solid Army Caps-Black W35S45D in the past week. \n'